In [30]:
import torch
import numpy as np
import random

from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import LinearLR
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ucimlrepo import fetch_ucirepo 

from models import MLP

seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ("Device: ", device)
print ("Seed: ", seed)

# set random seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

Device:  cuda
Seed:  42


In [45]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features.to_numpy()
y = wine_quality.data.targets.to_numpy()

# train test split
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

# standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# summary of the dataset
print ("X_train shape: ", X_train.shape)
print ("y_train shape: ", y_train.shape)
print ("X_test shape: ", X_test.shape)
print ("y_test shape: ", y_test.shape)

# create dataloaders
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
print ("Batch size: ", batch_size)
print ("Number of iterations per epoch:", len(train_dataloader))

X_train shape:  torch.Size([5197, 11])
y_train shape:  torch.Size([5197, 1])
X_test shape:  torch.Size([1300, 11])
y_test shape:  torch.Size([1300, 1])
Batch size:  128
Number of iterations per epoch: 41


In [57]:
epochs = 400 # 100, 200, 300, 400, 500
lr = 1e-3 # 1e-2. 1e-3. 1e-4
model = MLP(X_train.shape[-1], [32, 32], 1).to(device)
optimizer = Adam(model.parameters(), lr=lr)
lr_scheduler = LinearLR(optimizer, end_factor=0, start_factor=1, total_iters=epochs*len(train_dataloader))

In [58]:
# training loop

train_losses = []
test_losses = []

for epoch in range(epochs):

    model.train()
    train_loss = 0

    for i, (X_batch, y_batch) in enumerate(train_dataloader):
        y_pred = model(X_batch)
        loss = torch.nn.functional.mse_loss(y_pred, y_batch, reduction='mean')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        train_loss += loss.item()
    
    train_losses.append(train_loss/len(train_dataloader))

    model.eval()
    y_pred = model(X_test)
    test_loss = torch.mean((y_pred - y_test)**2).item()
    test_losses.append(test_loss)

    if epoch % (epochs//10) == 0:
        print (f"Epoch: {epoch} Train Loss: {train_loss/len(train_dataloader)} Test Loss: {test_loss}")

Epoch: 0 Train Loss: 30.31999285628156 Test Loss: 23.968900680541992
Epoch: 40 Train Loss: 0.4783033888514449 Test Loss: 0.5136999487876892
Epoch: 80 Train Loss: 0.4447713472494265 Test Loss: 0.4837651252746582
Epoch: 120 Train Loss: 0.42693787231677915 Test Loss: 0.4745670258998871
Epoch: 160 Train Loss: 0.41273760359461714 Test Loss: 0.47068271040916443
Epoch: 200 Train Loss: 0.4040468051666167 Test Loss: 0.4673764407634735
Epoch: 240 Train Loss: 0.39574643824158645 Test Loss: 0.4680960774421692
Epoch: 280 Train Loss: 0.3879641883256959 Test Loss: 0.46133914589881897
Epoch: 320 Train Loss: 0.38429050329254894 Test Loss: 0.4621370732784271
Epoch: 360 Train Loss: 0.3809779860624453 Test Loss: 0.46105605363845825
